***
# Final Project Milestone #3: Data Cleaning 
### Felicia Faith Hangge
### 29 April 2025

#### Analyzing Citi Bike usage trends in New York City to assist Citi Bike and their parent company Lyft make smarter bussiness decisions by providing insights on when, where, and how people use their bikes.  

##### Data Source: https://citibikenyc.com/system-data (202410-citibike-tripdata_1.csv)

***

## 1.) Importing Dataset + Libraries 
***

In [7]:
import pandas as pd

In [9]:
file_path = "202410-citibike-tripdata_1.csv"
df = pd.read_csv('/Users/faitth/Desktop/CMP-262/Final Project/202410-citibike-tripdata_1.csv')

/var/folders/6t/l24_hh0d7rddqfw4j3qphcfw0000gn/T/ipykernel_19004/1791134792.py:2: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/faitth/Desktop/CMP-262/Final Project/202410-citibike-tripdata_1.csv')


In [79]:
dt_fix = {
    'start_station_id': 'str',
    'end_station_id': 'str' }
file_path = "202410-citibike-tripdata_1.csv"
df = pd.read_csv('/Users/faitth/Desktop/CMP-262/Final Project/202410-citibike-tripdata_1.csv',
                dtype = dt_fix,
                low_memory=False)

In [80]:
df.shape 

(1000000, 13)

## 1.) Fixing Column Names + Formatting 
***

#### Renaming columns for easier use 

In [85]:
df.rename(columns={
    'ride_id': 'RideID',
    'rideable_type': 'BikeType',
    'started_at': 'StartTime',
    'ended_at': 'EndTime',
    'start_station_name': 'StartStation',
    'start_station_id': 'StartStationID',
    'end_station_name': 'EndStation',
    'end_station_id': 'EndStationID',
    'start_lat': 'StartLat',
    'start_lng': 'StartLng',
    'end_lat': 'EndLat',
    'end_lng': 'EndLng',
    'member_casual': 'UserType' }, 
          inplace=True)

In [87]:
df.columns

Index(['RideID', 'BikeType', 'StartTime', 'EndTime', 'StartStation',
       'StartStationID', 'EndStation', 'EndStationID', 'StartLat', 'StartLng',
       'EndLat', 'EndLng', 'UserType'],
      dtype='object')

## 2.) Investigating Missing Values (NaNs) 
***

#### Counting missing values per column 

In [92]:
df.isnull().sum()

RideID               0
BikeType             0
StartTime            0
EndTime              0
StartStation       283
StartStationID     283
EndStation        1341
EndStationID      1460
StartLat             0
StartLng             0
EndLat             104
EndLng             104
UserType             0
dtype: int64

#### Should these be replaced or deleted? 
##### No, these columns are needed to calculate the starting and ending points of trips but since the amount of NaN values is very small compared to the total dataset (1,460 / 1,000,000 = .146% of all data) it would be safe to remove the rows with NaN values instead of replace them with other values that can skew analysis. 

#### Dropping rows with NaN values 

In [102]:
df.dropna(subset =[
    'StartStation', 
    'StartStationID',
    'EndStation',
    'EndStationID',
    'EndLat',
    'EndLng' ],
          inplace = True)

In [67]:
df.isnull().sum()

RideID            0
BikeType          0
StartTime         0
EndTime           0
StartStation      0
StartStationID    0
EndStation        0
EndStationID      0
StartLat          0
StartLng          0
EndLat            0
EndLng            0
UserType          0
dtype: int64

***
## 3.) Assessing Outliers 
***

In [52]:
df.describe()

,StartLat,StartLng,EndLat,EndLng
count,998262.000000,998262.000000,998262.000000,998262.000000
mean,40.737541,-73.971116,40.736877,-73.969972
std,0.041185,0.029430,0.163222,0.288246
min,40.633385,-74.026823,0.000000,-74.067622
25%,40.711863,-73.991930,40.711066,-73.992085
50%,40.736592,-73.979069,40.736245,-73.979294
75%,40.761573,-73.954692,40.761712,-73.954926
max,40.886300,-73.846720,40.886300,0.000000


#### Cleaning invalid coordinate data
##### While looking at the statistical summaries for the columns, there are clear outliers in the EndLat and EndLng columns where the values are 0, which is not a valid location. These columns contain values that represent the latitude and longitude coordinates of each ride, since the coordinates (0,0) are obviously not in New York City, they are invalid when dealing with Citi Bike Trips.

#### Removing any rows where values for StartLat, StartLng, EndLat, and EndLng are equal to 0.

In [105]:
df = df[(df['StartLat'] != 0) & (df['StartLng'] != 0) &
        (df['EndLat'] != 0) & (df['EndLng'] != 0)]

In [107]:
df.describe()

,StartLat,StartLng,EndLat,EndLng
count,998247.000000,998247.000000,998247.000000,998247.000000
mean,40.737541,-73.971117,40.737489,-73.971083
std,0.041185,0.029429,0.041296,0.029463
min,40.633385,-74.026823,40.633385,-74.067622
25%,40.711863,-73.991930,40.711066,-73.992085
50%,40.736592,-73.979069,40.736245,-73.979294
75%,40.761573,-73.954692,40.761712,-73.954926
max,40.886300,-73.846720,40.886300,-73.846720


#### Cleaning invalid ride data
##### To easier analyze the behavior of riders, I created a new column titled RideDurationMin to measure the total duration of each trip in minutes.

##### Convert StartTime and EndTime from string to datetime in order to perform calculations

In [116]:
df['StartTime'] = pd.to_datetime(df['StartTime'])
df['EndTime'] = pd.to_datetime(df['EndTime'])

##### Create new column 

In [118]:
df['RideDurationMin'] = (df['EndTime'] - df['StartTime']).dt.total_seconds() / 60

In [122]:
df.sample(1)

,RideID,BikeType,StartTime,EndTime,StartStation,StartStationID,EndStation,EndStationID,StartLat,StartLng,EndLat,EndLng,UserType,RideDurationMin
325831,E8672D08A2774BE6,electric_bike,2024-10-01 14:49:15.390,2024-10-01 15:09:41.256,E 17 St & Broadway,5980.10,E 82 St & East End Ave,7049.04,40.737006,-73.990134,40.772461,-73.946821,member,20.4311


#### Checking for outliers in RideDurationMin

In [140]:
df.describe()

,StartTime,EndTime,StartLat,StartLng,EndLat,EndLng,RideDurationMin
count,998247,998247,998247.000000,998247.000000,998247.000000,998247.000000,998247.000000
mean,2024-10-07 21:24:13.362079744,2024-10-07 21:37:19.909940224,40.737541,-73.971117,40.737489,-73.971083,13.109131
min,2024-09-30 11:05:33.619000,2024-10-01 00:00:02.702000,40.633385,-74.026823,40.633385,-74.067622,1.001233
25%,2024-10-04 13:21:48.310999808,2024-10-04 13:34:38.769499904,40.711863,-73.991930,40.711066,-73.992085,5.379808
50%,2024-10-07 18:48:17.024000,2024-10-07 19:00:55.156000,40.736592,-73.979069,40.736245,-73.979294,9.339333
75%,2024-10-11 09:19:21.993499904,2024-10-11 09:30:53.420499968,40.761573,-73.954692,40.761712,-73.954926,16.160150
max,2024-10-14 19:59:59.570000,2024-10-15 13:37:17.976000,40.886300,-73.846720,40.886300,-73.846720,1498.964883
std,NaN,NaN,0.041185,0.029429,0.041296,0.029463,19.188829


##### Removing any rides that are unusually short (under 2 minutes) or long (longer than 6 hours = 360 minutes) 

In [164]:
df = df[(df['RideDurationMin'] >= 2) & (df['RideDurationMin'] <= 360)]

In [166]:
df.describe()

,StartTime,EndTime,StartLat,StartLng,EndLat,EndLng,RideDurationMin
count,914351,914351,914351.000000,914351.000000,914351.000000,914351.000000,914351.000000
mean,2024-10-07 21:27:45.422510592,2024-10-07 21:41:36.470227968,40.737293,-73.971545,40.737232,-73.971505,13.850795
min,2024-09-30 18:56:30.485000,2024-10-01 00:00:02.702000,40.633385,-74.026823,40.633385,-74.067622,3.000000
25%,2024-10-04 13:37:46.984499968,2024-10-04 13:52:27.494000128,40.711863,-73.992031,40.710709,-73.992131,6.262817
50%,2024-10-07 18:42:53.550000128,2024-10-07 18:56:17.372999936,40.736529,-73.979069,40.735877,-73.979504,10.169633
75%,2024-10-11 09:40:16.659500032,2024-10-11 09:52:20.680999936,40.761294,-73.955156,40.761712,-73.955637,17.048033
max,2024-10-14 19:59:59.570000,2024-10-14 22:27:45.848000,40.886300,-73.846720,40.886300,-73.846720,356.487733
std,NaN,NaN,0.040808,0.029127,0.040934,0.029169,13.311635


***
## 3.) Saving Cleaned Data 
***

In [ ]:
df.to_csv('cleaned_citibike_oct2024.csv', index=False)

***